In [14]:
import warnings
warnings.filterwarnings("ignore")
from keras.utils.np_utils import to_categorical
from tensorflow.keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Concatenate
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam

import re
np.random.seed(5)
import os

In [15]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.metrics import log_loss, classification_report

In [16]:
# Análise de reviews do Smartwatch Amazfit Bip U Pro - Preto -  Amazon
df_texto = pd.read_csv('run_results2.csv')
df_texto.head(2)

,controle_review_curto,controle_review_longo,controle_nota,controle_nota_url,controle_selection1
0,Mais que recomendo!!!!,Que sensacional! O modelo da minha tv é uma da...,"5,0 de 5 estrelas",https://www.amazon.com.br/gp/customer-reviews/...,Avaliado no Brasil em 10 de maio de 2021
1,Não cumpre plenamente o que se propõe,Devo dizer que estou bem decepcionado.\nCompre...,"2,0 de 5 estrelas",https://www.amazon.com.br/gp/customer-reviews/...,Avaliado no Brasil em 13 de julho de 2021


In [3]:
df_texto.tail(2)

,controle_review_curto,controle_review_longo,controle_nota,controle_nota_url,controle_selection1
1508,Funcionalidades confirmadas.,O aparelho é realmente aquilo que esta descrit...,"5,0 de 5 estrelas",https://www.amazon.com.br/gp/customer-reviews/...,Avaliado no Brasil em 14 de fevereiro de 2022
1509,Não vale a pena.,Aparente parece um ótimo produto. Porém tem mu...,"1,0 de 5 estrelas",https://www.amazon.com.br/gp/customer-reviews/...,Avaliado no Brasil em 27 de janeiro de 2022


In [4]:
df_texto.drop('controle_nota_url', axis=1, inplace= True)
df_texto.head(2)

,controle_review_curto,controle_review_longo,controle_nota,controle_selection1
0,Mais que recomendo!!!!,Que sensacional! O modelo da minha tv é uma da...,"5,0 de 5 estrelas",Avaliado no Brasil em 10 de maio de 2021
1,Não cumpre plenamente o que se propõe,Devo dizer que estou bem decepcionado.\nCompre...,"2,0 de 5 estrelas",Avaliado no Brasil em 13 de julho de 2021


In [5]:
# Pré-processando os dados de datas.
meses=['janeiro',' fevereiro', 'março', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro']
mesess = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
numeros=['0','1','2','3','4','5','6','7','8','9']
i=0
df_texto['data']=1

for datas in df_texto['controle_selection1']:
    if datas[23] in numeros:
        dia = datas[22:24]
    else:
        dia = datas[22]
    for mese in meses:
        if datas[28-len(datas):-8] in mese:
            mes = mesess[meses.index(mese)]
    ano = datas[-4:len(datas)]
    if '202' in datas:
        if '2021' in datas:
            ano = '2021'
        elif '2022' in datas:
            ano = '2022'
    
    df_texto['data'].iloc[i,:]=dia + "-" + mes + "-" + ano
    i=i+1

C:\Users\wfs4.NB555-WFS4\anaconda3\lib\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value
C:\Users\wfs4.NB555-WFS4\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [6]:
df_texto.head(60)

,controle_review_curto,controle_review_longo,controle_nota,controle_selection1,data
0,Mais que recomendo!!!!,Que sensacional! O modelo da minha tv é uma da...,"5,0 de 5 estrelas",Avaliado no Brasil em 10 de maio de 2021,10-05-2021
1,Não cumpre plenamente o que se propõe,Devo dizer que estou bem decepcionado.\nCompre...,"2,0 de 5 estrelas",Avaliado no Brasil em 13 de julho de 2021,13-07-2021
2,Não tem integração com toda TV,Eu comprei esse modelo justamente pelos botões...,"3,0 de 5 estrelas",Avaliado no Brasil em 15 de maio de 2021,15-05-2021
3,"O PRODUTO É ÓTIMO, MAS A ENTREGA FOI MEDONHA!!!","Pow, Amazon, que isso? vcs eram tão competente...","1,0 de 5 estrelas",Avaliado no Brasil em 27 de setembro de 2021,27-09-2021
4,Versátil,Pontos positivos: o processo de instalação é b...,"5,0 de 5 estrelas",Avaliado no Brasil em 17 de maio de 2021,17-05-2021
5,Muito Bom e rápido!,"Teclado na tela não é QWERT, e também o navega...","4,0 de 5 estrelas",Avaliado no Brasil em 7 de maio de 2021,7-05-2021
6,"ótimo produto, péssima logística para entrega ...","Gostei muito do produto, achei somente que o c...","1,0 de 5 estrelas",Avaliado no Brasil em 18 de agosto de 2021,18-08-2021
7,A Alexa não nos escuta se não apertamos o botã...,Sempre via em vídeos o pessoal falando com a A...,"5,0 de 5 estrelas",Avaliado no Brasil em 23 de outubro de 2021,23-10-2021
8,Fire Stick trava e conexão com Echo Studio é p...,O Fire Stick trava (aconteceu várias vezes) e ...,"3,0 de 5 estrelas",Avaliado no Brasil em 4 de julho de 2021,4-07-2021
9,ÓTIMA ESCOLHA!,Muito contente com o FIRE TV STICK da Amazon. ...,"5,0 de 5 estrelas",Avaliado no Brasil em 16 de novembro de 2021,16-11-2021


In [7]:
i=0
df_texto['nota']=1
for notas in df_texto['controle_nota']:
    df_texto['nota'].iloc[i,:]=notas[:1]
    i=i+1
    
df_texto.head(10)

C:\Users\wfs4.NB555-WFS4\anaconda3\lib\site-packages\pandas\core\indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value


,controle_review_curto,controle_review_longo,controle_nota,controle_selection1,data,nota
0,Mais que recomendo!!!!,Que sensacional! O modelo da minha tv é uma da...,"5,0 de 5 estrelas",Avaliado no Brasil em 10 de maio de 2021,10-05-2021,5
1,Não cumpre plenamente o que se propõe,Devo dizer que estou bem decepcionado.\nCompre...,"2,0 de 5 estrelas",Avaliado no Brasil em 13 de julho de 2021,13-07-2021,2
2,Não tem integração com toda TV,Eu comprei esse modelo justamente pelos botões...,"3,0 de 5 estrelas",Avaliado no Brasil em 15 de maio de 2021,15-05-2021,3
3,"O PRODUTO É ÓTIMO, MAS A ENTREGA FOI MEDONHA!!!","Pow, Amazon, que isso? vcs eram tão competente...","1,0 de 5 estrelas",Avaliado no Brasil em 27 de setembro de 2021,27-09-2021,1
4,Versátil,Pontos positivos: o processo de instalação é b...,"5,0 de 5 estrelas",Avaliado no Brasil em 17 de maio de 2021,17-05-2021,5
5,Muito Bom e rápido!,"Teclado na tela não é QWERT, e também o navega...","4,0 de 5 estrelas",Avaliado no Brasil em 7 de maio de 2021,7-05-2021,4
6,"ótimo produto, péssima logística para entrega ...","Gostei muito do produto, achei somente que o c...","1,0 de 5 estrelas",Avaliado no Brasil em 18 de agosto de 2021,18-08-2021,1
7,A Alexa não nos escuta se não apertamos o botã...,Sempre via em vídeos o pessoal falando com a A...,"5,0 de 5 estrelas",Avaliado no Brasil em 23 de outubro de 2021,23-10-2021,5
8,Fire Stick trava e conexão com Echo Studio é p...,O Fire Stick trava (aconteceu várias vezes) e ...,"3,0 de 5 estrelas",Avaliado no Brasil em 4 de julho de 2021,4-07-2021,3
9,ÓTIMA ESCOLHA!,Muito contente com o FIRE TV STICK da Amazon. ...,"5,0 de 5 estrelas",Avaliado no Brasil em 16 de novembro de 2021,16-11-2021,5


In [8]:
df_texto.dtypes

controle_review_curto    object
controle_review_longo    object
controle_nota            object
controle_selection1      object
data                     object
nota                      int64
dtype: object

In [9]:
df = df_texto[['controle_review_longo','data','nota']]
df.head(10)

,controle_review_longo,data,nota
0,Que sensacional! O modelo da minha tv é uma da...,10-05-2021,5
1,Devo dizer que estou bem decepcionado.\nCompre...,13-07-2021,2
2,Eu comprei esse modelo justamente pelos botões...,15-05-2021,3
3,"Pow, Amazon, que isso? vcs eram tão competente...",27-09-2021,1
4,Pontos positivos: o processo de instalação é b...,17-05-2021,5
5,"Teclado na tela não é QWERT, e também o navega...",7-05-2021,4
6,"Gostei muito do produto, achei somente que o c...",18-08-2021,1
7,Sempre via em vídeos o pessoal falando com a A...,23-10-2021,5
8,O Fire Stick trava (aconteceu várias vezes) e ...,4-07-2021,3
9,Muito contente com o FIRE TV STICK da Amazon. ...,16-11-2021,5


In [10]:
# Transformando tipo de dados das colunas
df['data'] = pd.to_datetime(df['data'], format="%d-%m-%Y")
df['nota'] =  df['nota'].astype(int)
df.dtypes

C:\Users\WFS4~1.NB5\AppData\Local\Temp/ipykernel_14420/2103150511.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['data'] = pd.to_datetime(df['data'], format="%d-%m-%Y")
C:\Users\WFS4~1.NB5\AppData\Local\Temp/ipykernel_14420/2103150511.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['nota'] =  df['nota'].astype(int)


controle_review_longo            object
data                     datetime64[ns]
nota                              int32
dtype: object

In [11]:
df.tail(30)

,controle_review_longo,data,nota
1480,Só não dou 5 estrela pq infelizmente não tem o...,2021-12-27,4
1481,Gostaria de ter um controle que possa ter aces...,2021-10-17,2
1482,O Fire Stick TV é o melhor para quem quer ter ...,2022-01-25,5
1483,"Imagem excelente, fácil de instalar, áudio bom...",2021-05-08,5
1484,"Simplesmente sensacional, chegou antes da Data...",2022-01-21,5
1485,Gostei muito do produto. Rápido e intuitivo.\n...,2021-05-11,4
1486,ótimo aparelho. Apesar de não ter o app da Glo...,2022-01-02,3
1487,Pra quem não quer comprar uma tv nova essa é u...,2022-01-23,5
1488,"De vez em quando dá uma travadinha, o que pode...",2021-10-11,4
1489,"Eu amei o fire stick, só tive problemas em ach...",2022-01-30,5


In [12]:
# Separando dados por niveis
escala = [
    (df["nota"] < 3),
    (df["nota"] == 3),
    (df["nota"] >= 4)
         ]
nivel = ["Negativo", "Neutro", "Positivo"]

df["Notas"] = np.select(escala, nivel)
df.head()

C:\Users\WFS4~1.NB5\AppData\Local\Temp/ipykernel_14420/3262635037.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Notas"] = np.select(escala, nivel)


,controle_review_longo,data,nota,Notas
0,Que sensacional! O modelo da minha tv é uma da...,2021-05-10,5,Positivo
1,Devo dizer que estou bem decepcionado.\nCompre...,2021-07-13,2,Negativo
2,Eu comprei esse modelo justamente pelos botões...,2021-05-15,3,Neutro
3,"Pow, Amazon, que isso? vcs eram tão competente...",2021-09-27,1,Negativo
4,Pontos positivos: o processo de instalação é b...,2021-05-17,5,Positivo


#  Aplicando CNN

Preprarando os dados de reviews

In [13]:
# Convertendo as avaliações
def converteCategoria(df, coluna):
    le = preprocessing.LabelEncoder()
    le.fit(df[coluna])
    df[coluna] = le.transform(df[coluna])
    return le

labelEncoderNotas = converteCategoria(df, 'Notas')
df.head()

C:\Users\WFS4~1.NB5\AppData\Local\Temp/ipykernel_14420/1684540264.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[coluna] = le.transform(df[coluna])


,controle_review_longo,data,nota,Notas
0,Que sensacional! O modelo da minha tv é uma da...,2021-05-10,5,2
1,Devo dizer que estou bem decepcionado.\nCompre...,2021-07-13,2,0
2,Eu comprei esse modelo justamente pelos botões...,2021-05-15,3,1
3,"Pow, Amazon, que isso? vcs eram tão competente...",2021-09-27,1,0
4,Pontos positivos: o processo de instalação é b...,2021-05-17,5,2


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

tokenized_sents = [word_tokenize(i) for i in df['controle_review_longo'].values]

from gensim.models import Word2Vec
model_review = Word2Vec(tokenized_sents, size=100, window=5, min_count=1, workers=4)

In [ ]:
X, y = [], []
max_len = 0

for i, row in reviews_df.iterrows():
    vetores = model_review[word_tokenize(row['review'])]

    max_len = max(max_len, len(vetores))
    X.append(vetores)
    y.append(row['rating']-1)
print("max_len: {}".format(max_len))

In [ ]:
def transform(exemplos, dimension):
    results = np.zeros((len(exemplos), dimension, 100))
    for i, sequence in enumerate(exemplos):
        results[i, :len(sequence), :] = sequence
    return results
 
X = transform(X, max_len)
y = np.array(y).astype("float32")

In [ ]:
X.shape

In [ ]:
np.unique(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout

model = Sequential()
model.add(Flatten(input_shape=(max_len, 100)))
model.add(Dense(1000, activation='relu')),
model.add(Dropout(0.3))
model.add(Dense(1000, activation='relu')),
model.add(Dropout(0.3))
model.add(Dense(1000, activation='relu')),
model.add(Dense(5, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer = "adam",
              loss = "sparse_categorical_crossentropy",
              metrics = ["accuracy"]
              )

In [ ]:
results = model.fit(X_train, y_train,
                    epochs= 5,
                    batch_size = 100,
                    validation_data = (X_test, y_test)
                    )